## Collecting Date 20 frames

In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [4]:
# mp holistic for keypoints
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities


# functions for drawing landmarks
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results


# drawing landmarks
def draw_styled_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

    
# preparing vector of date
from numpy import zeros
def extract_keypoints(results):
    # for pose - FACE INCLUDED
    if (results.pose_landmarks == None):
        face = []
        pose_left = []
        pose_right = []
    else:
        x = 0
        y = 0
        #z = 0
        for i in range(11):
            if results.pose_landmarks.landmark[i].visibility > 0.95:
                x += results.pose_landmarks.landmark[i].x
                y += results.pose_landmarks.landmark[i].y
                #z += results.pose_landmarks.landmark[i].z
            else:
                return [], [], [], [], [], # no mercy jak nie ma jakiegos puntku na twarzy to jest nasz punkt odniesienia
        x = x/11
        y = y/11
        #z = z/11
        face = np.array([x,y])
        #face = np.array([x,y,z])
        
        #meme = []
        pose_left = []
        pose_right = []
        
        # middle point in chest
        if(results.pose_landmarks.landmark[11].visibility > 0.85 and results.pose_landmarks.landmark[12].visibility > 0.85):
            pose_left.append(((results.pose_landmarks.landmark[11].x + results.pose_landmarks.landmark[12].x) /2) - face[0])
            pose_left.append(((results.pose_landmarks.landmark[11].y + results.pose_landmarks.landmark[12].y) /2) - face[1])
            pose_right.append(((results.pose_landmarks.landmark[11].x + results.pose_landmarks.landmark[12].x) /2) - face[0])
            pose_right.append(((results.pose_landmarks.landmark[11].y + results.pose_landmarks.landmark[12].y) /2) - face[1])
        else:
            pose_left.append(0)
            pose_left.append(0)
            pose_right.append(0)
            pose_right.append(0)

        for i in range(11, 17):
            if(results.pose_landmarks.landmark[i].visibility > 0.85): # to jeszcze do empirycznego ustawienia !!!!!!!!
                if(i % 2 == 0):
                    pose_right.append(results.pose_landmarks.landmark[i].x - face[0])
                    pose_right.append(results.pose_landmarks.landmark[i].y - face[1])
                else:     
                    pose_left.append(results.pose_landmarks.landmark[i].x - face[0])
                    pose_left.append(results.pose_landmarks.landmark[i].y - face[1])
                
                #meme.append(results.pose_landmarks.landmark[i].visibility)
            else:
                if(i % 2 == 0):
                    pose_right.append(0)
                    pose_right.append(0) 
                else:
                    pose_left.append(0)
                    pose_left.append(0)     
                                      
                #meme.append(-1 * results.pose_landmarks.landmark[i].visibility)

        pose_left = np.array(pose_left)
        pose_right = np.array(pose_right)
        #vis = np.array(meme)

    # for right hand
    if (results.right_hand_landmarks == None):
        right_hand = []
    else:
        right_hand = np.array([[-1 * (res.x - face[0]), res.y - face[1]] for res in results.right_hand_landmarks.landmark]).flatten()

    # for left hand
    if (results.left_hand_landmarks == None):
        left_hand = []
    else:
        left_hand = np.array([[res.x - face[0], res.y - face[1]] for res in results.left_hand_landmarks.landmark]).flatten()
    
    return face, pose_left, pose_right, left_hand, right_hand
    
    
# making folders
no_sequences = 50 # 50 videos
sequence_length = 12 # for 20 frames
actions = np.array(["",'hello','eat','I_me','need','thanks', 'drink', 'beer'])
    
DATA_PATH = os.path.join('MP_Data') 
for action in actions:
        try:       
            os.makedirs(os.path.join(DATA_PATH, action))
        except:
            pass

In [6]:
import shutil

out = False
go = False
fixing = False

cap = cv2.VideoCapture(0)
ret, image = cap.read()
cv2.namedWindow("OpenCV date_set", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("OpenCV date_set",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_NORMAL)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.4, min_tracking_confidence=0.4) as holistic:
    # Loop through actions
    for action in actions:
        if (out):
            break
        tmp = image.copy()
        cv2.putText(tmp, "slowo: '{}' ".format(action), (200,200), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 4, cv2.LINE_AA)
        cv2.imshow('OpenCV date_set', tmp)
        cv2.waitKey(1500)
        
        # Loop through num of videos per sign
        sequence = 0
        while(sequence < no_sequences):
            sequence += 1
            if((sequence-1)%5 == 0 and not fixing): # break for rest after some frames
                print(action, sequence) # info
                tmp = image.copy()
                cv2.putText(tmp, "kontynuj {} '{}' wcisnij 's'".format(sequence, action), (25,150), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255 , 0, 0), 4, cv2.LINE_AA)
                cv2.putText(tmp, "powtorz 5 ostatnich znakow wcisnij 'b'".format(action), (25,200), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 4, cv2.LINE_AA)
                cv2.putText(tmp, "przewin 5 kolejnych znakow wcisnij 'c'".format(action), (25,250), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 4, cv2.LINE_AA)
                cv2.putText(tmp, "przeskocz robienie tego slowa wcisnij 'q'".format(action), (25,300), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 4, cv2.LINE_AA)
                cv2.putText(tmp, "wyjdz z programu 'x'".format(action), (50,425), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 4, cv2.LINE_AA)
                cv2.imshow('OpenCV date_set', tmp)
                while True: 
                    # pause on sreen
                    tmp = cv2.waitKey(1000)
                    if tmp == ord('s'):
                        if(os.path.isdir(os.path.join(DATA_PATH, action, str(sequence)))):
                            tmp = image.copy()
                            cv2.putText(tmp, "masz balagan! istnieje juz plik {}".format(sequence), (50,200), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0 , 0, 255), 4, cv2.LINE_AA)
                            cv2.putText(tmp, "usun go recznie albo skipnij do kolejnych", (25,250), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0 , 0, 255), 4, cv2.LINE_AA)
                            cv2.imshow('OpenCV date_set', tmp)
                            cv2.waitKey(2000)
                            
                            tmp = image.copy()
                            cv2.putText(tmp, "kontynuj {} '{}' wcisnij 's'".format(sequence, action), (25,150), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255 , 0, 0), 4, cv2.LINE_AA)
                            cv2.putText(tmp, "powtorz 5 ostatnich znakow wcisnij 'b'".format(action), (25,200), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 4, cv2.LINE_AA)
                            cv2.putText(tmp, "przewin 5 kolejnych znakow wcisnij 'c'".format(action), (25,250), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 4, cv2.LINE_AA)
                            cv2.putText(tmp, "przeskocz robienie tego slowa wcisnij 'q'".format(action), (25,300), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 4, cv2.LINE_AA)
                            cv2.putText(tmp, "wyjdz z programu 'x'".format(action), (50,425), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 4, cv2.LINE_AA)
                            cv2.imshow('OpenCV date_set', tmp)  
                        else:
                            #stop before start
                            cv2.putText(image, 'BREAK !', (250,200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0 ,0, 255), 4, cv2.LINE_AA)
                            cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence-1), (15,15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, cv2.LINE_AA)
                            cv2.putText(image, "skip 'z'", (15,30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, cv2.LINE_AA)
                            # Show to screen
                            cv2.imshow('OpenCV date_set', image)
                            cv2.waitKey(500)
                            break
                    elif tmp == ord('b'):
                        print("DELETE")
                        if(sequence > 5):
                            sequence -= 5              
                            for i in range(5):
                                if(os.path.isdir(os.path.join(DATA_PATH, action, str(sequence + i)))):
                                    shutil.rmtree(os.path.join(DATA_PATH, action, str(sequence + i)))
                                
                            print(action, sequence) # info
                            tmp = image.copy()
                            cv2.putText(tmp, "kontynuj {} '{}' wcisnij 's'".format(sequence, action), (25,150), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255 , 0, 0), 4, cv2.LINE_AA)
                            cv2.putText(tmp, "powtorz 5 ostatnich znakow wcisnij 'b'".format(action), (25,200), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 4, cv2.LINE_AA)
                            cv2.putText(tmp, "przewin 5 kolejnych znakow wcisnij 'c'".format(action), (25,250), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 4, cv2.LINE_AA)
                            cv2.putText(tmp, "przeskocz robienie tego slowa wcisnij 'q'".format(action), (25,300), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 4, cv2.LINE_AA)
                            cv2.putText(tmp, "wyjdz z programu 'x'".format(action), (50,425), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 4, cv2.LINE_AA)
                            cv2.imshow('OpenCV date_set', tmp)  
                    elif tmp == ord('c'):
                        print("CONTINUE")
                        if(sequence < 45):
                            sequence += 5
          
                        print(action, sequence) # info
                        tmp = image.copy()
                        cv2.putText(tmp, "kontynuj {} '{}' wcisnij 's'".format(sequence, action), (25,150), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255 , 0, 0), 4, cv2.LINE_AA)
                        cv2.putText(tmp, "powtorz 5 ostatnich znakow wcisnij 'b'".format(action), (25,200), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 4, cv2.LINE_AA)
                        cv2.putText(tmp, "przewin 5 kolejnych znakow wcisnij 'c'".format(action), (25,250), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 4, cv2.LINE_AA)
                        cv2.putText(tmp, "przeskocz robienie tego slowa wcisnij 'q'".format(action), (25,300), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 4, cv2.LINE_AA)
                        cv2.putText(tmp, "wyjdz z programu 'x'".format(action), (50,425), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 4, cv2.LINE_AA)
                        cv2.imshow('OpenCV date_set', tmp)                   
                    elif tmp == ord('q'):
                        print("NEXT")
                        go = True
                        break 
                    elif tmp == ord('x'):
                        print("koniec")
                        out = True
                        break
                if(out or go):
                    go = False
                    break
            
            # creating file
            if os.listdir(os.path.join(DATA_PATH, action)):
                dir_list = list(map(int, os.listdir(os.path.join(DATA_PATH, action))))
                dir_list.sort()
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))  # bo zmiany nazw plikow
                        
            #stop before start
            cv2.putText(image, 'BREAK !', (250,200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0 ,0, 255), 4, cv2.LINE_AA)
            cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence-1), (15,15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, "skip 'z'", (15,30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, cv2.LINE_AA)
            cv2.imshow('OpenCV date_set', image)
            cv2.waitKey(1000)
            
            # Loop through video length aka sequence length
            blank_pose_left = 0
            all_frames_pose_left = []
            blank_pose_right = 0
            all_frames_pose_right = []
            blank_left_hand = 0
            all_frames_left_hand = []
            blank_right_hand = 0
            all_frames_right_hand = []
            
            frame_num = 0
            while(frame_num != sequence_length):
                frame_num += 1
                
                # Read feed
                ret, frame = cap.read()
                
                ## Making contrast i can put later to mediapipe detection to not have contrast all the time on video
                lab= cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
                l_channel, a, b = cv2.split(lab)
                # Applying CLAHE to L-channel
                # feel free to try different values for the limit and grid size:
                clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
                cl = clahe.apply(l_channel)
                # merge the CLAHE enhanced L-channel with the a and b channel
                limg = cv2.merge((cl,a,b))
                # Converting image from LAB Color model to BGR color spcae
                frame = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
                # print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # show wchich video
                tmp = image.copy()
                cv2.putText(tmp, 'GO {}'.format(sequence_length - frame_num), (300,200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 1, cv2.LINE_AA)
                cv2.putText(tmp, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(tmp, "skip 'z'", (15,30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, cv2.LINE_AA)
                
                # Show to screen
                cv2.imshow('OpenCV date_set', tmp)
                
                # extracting keypoints
                face, pose_left, pose_right, left_hand, right_hand = extract_keypoints(results)   
                if len(face) == 0:
                    print("nic nie ma na ekraie")
                    frame_num = sequence_length
                else:
                    # prawe i lewe cialko tez trzeba przebadac
                    if len(left_hand) == 0:
                        left_hand = np.zeros(42)
                        #print("nie ma lewej dloni na ekranie")
                        # zliczamy
                        blank_left_hand += 1
                    if len(right_hand) == 0:
                        right_hand = np.zeros(42)
                        #print("nie ma prawej dloni na ekranie")
                        #zliaczamy
                        blank_right_hand += 1
                    if len(pose_left) - np.count_nonzero(pose_left) != 0:
                        #print("nie ma lewego ramienia")
                        #zliczamy
                        blank_pose_left += 1
                    if len(pose_right) - np.count_nonzero(pose_right) != 0:
                        #print("nie ma prawego ramienia")
                        #zliczamy
                        blank_pose_right += 1
                        
                    # pierwsza klatka musi miec wartosci chociaz jedna reke i ramie
                    if (frame_num == 1 and blank_left_hand + blank_pose_left != 0 and blank_right_hand + blank_pose_right != 0):
                        print("Nie zlapano pierwszej klatki")
                        frame_num = sequence_length
                        blank_left_hand = 404
                    # cala historia robienia znaku
                    all_frames_pose_left.append(pose_left)
                    all_frames_pose_right.append(pose_right)
                    all_frames_left_hand.append(left_hand)
                    all_frames_right_hand.append(right_hand)
                        
                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('z'):
                    go = True
                    break
            if go:
                go = False
                fixing = False
                continue 
            
            
            # warunki  powtorzenia video
            if len(face) == 0: # jak nie ma face to nie ma ryja
                print("naprawiamy")
                print()
                shutil.rmtree(os.path.join(DATA_PATH, action, str(sequence)))
                
                tmp = image.copy()
                cv2.putText(tmp, "Nie ma nic na ekranie !", (100,200), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0 , 0, 255), 4, cv2.LINE_AA)
                cv2.putText(tmp, "trzeba powtorzyc nagranie {}".format(sequence), (100,250), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0 , 0, 255), 4, cv2.LINE_AA)
                cv2.putText(tmp, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(tmp, "skip 'z'", (15,30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, cv2.LINE_AA)
                cv2.imshow('OpenCV date_set', tmp)
                cv2.waitKey(2000)
                sequence -= 1
                fixing = True
            elif blank_left_hand == 404: # nie ma lewej ani prawej reki na pierwszej klatce
                print("naprawiamy")
                print()
                shutil.rmtree(os.path.join(DATA_PATH, action, str(sequence)))
                
                tmp = image.copy()
                cv2.putText(tmp, "BADZ gotowy na BREAKU !", (100,200), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0 , 0, 255), 4, cv2.LINE_AA)
                cv2.putText(tmp, "trzeba powtorzyc nagranie {}".format(sequence), (100,250), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0 , 0, 255), 4, cv2.LINE_AA)
                cv2.putText(tmp, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(tmp, "skip 'z'", (15,30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, cv2.LINE_AA)
                cv2.imshow('OpenCV date_set', tmp)
                cv2.waitKey(2000)
                sequence -= 1
                fixing = True  
            elif (blank_left_hand > 2 or blank_pose_left > 2) and (blank_right_hand > 2 or blank_pose_right > 2): # nie zebralo dobrze landmarkow
                print("za malo landmarkow")
                print("naprawiamy")
                print()
                shutil.rmtree(os.path.join(DATA_PATH, action, str(sequence)))
                
                tmp = image.copy()
                cv2.putText(tmp, "zlapano za malo landmarkow", (100,200), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0 , 0, 255), 4, cv2.LINE_AA)
                cv2.putText(tmp, "trzeba powtorzyc nagranie {}".format(sequence), (100,250), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0 , 0, 255), 4, cv2.LINE_AA)
                cv2.putText(tmp, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(tmp, "skip 'z'", (15,30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, cv2.LINE_AA)
                cv2.imshow('OpenCV date_set', tmp)
                cv2.waitKey(2000)
                sequence -= 1
                fixing = True  
            
            
            # zapis jesli wykryto ruch chociaz jednej reki
            else:                     
                print("blank_pose_left:", blank_pose_left, " blank_left_hand:",  blank_left_hand, "  blank_pose_right:", blank_pose_right,  " blank_right_hand:", blank_right_hand)                
                s1 = False
                s2 = False
                # czy jest z czego wyznaczac czy nie ma za duzo dziur dla lewej reki
                if blank_left_hand <= 2 and blank_pose_left <= 2:
                    varations = []
                    col = []
                    for it in range(42):
                        for frame in all_frames_left_hand:
                            col.append(10*frame[it])   # 1 dobrze wyznacza
                        # # eliminacja zer w wektorze dla hand
                        for it2 in range(1, len(col)):
                            if it2 == len(col)-1:
                                if(col[it2] == 0):
                                    col[it2] = col[it2-1]
                                    all_frames_left_hand[it2][it] = col[it2-1]
                            else:
                                if(col[it2] == 0):
                                    if(col[it2+1] == 0):
                                        col[it2] = col[it2-1]
                                        all_frames_left_hand[it2][it] = col[it2-1]
                                    else:
                                        col[it2] = (col[it2-1] + col[it2+1])/2
                                        all_frames_left_hand[it2][it] = (col[it2-1] + col[it2+1])/2y   
                        
                        # wyliczamy wariancje
                        srednio =  sum(col)/sequence_length
                        varations.append(np.sum( (col - srednio)**2 ) / len(col))
                        col = []
                    # decycja na podstawie wariancji 
                    if(np.sum(varations) >= 0.8):
                        print("suma: ", np.sum(varations), "lewa dlon sie porusza")
                        s1 = True
                    else:
                        print("suma: ", np.sum(varations), "lewa dlon sie nie poruszala")
                        
                    # # eliminacja zer w wektorze dla pose
                    col = []
                    for it in range(8):
                        for frame in all_frames_pose_left:
                            col.append(frame[it])   # 1 dobrze wyznacza

                        for it2 in range(1, len(col)):
                            if it2 == len(col)-1:
                                if(col[it2] == 0):
                                    col[it2] = col[it2-1]
                                    all_frames_pose_left[it2][it] = col[it2-1]
                            else:
                                if(col[it2] == 0):
                                    if(col[it2+1] == 0):
                                        col[it2] = col[it2-1]
                                        all_frames_pose_left[it2][it] = col[it2-1]
                                    else:
                                        col[it2] = (col[it2-1] + col[it2+1])/2
                                        all_frames_pose_left[it2][it] = (col[it2-1] + col[it2+1])/2
                        col = []
                else:
                    print("lewa reka jest podziurawiona albo nie ma jej w kadrze")
                    
                    
                # czy jest z czego wyznaczac czy nie ma za duzo dziur dla prawej reki
                if blank_right_hand <= 2 and blank_pose_right <= 2:
                    varations = []
                    col = []
                    for it in range(42):
                        for frame in all_frames_right_hand:
                            col.append(10*frame[it])   # 1 dobrze wyznacza
                        # # eliminacja zer w wektorze dla dloni
                        for it2 in range(1, len(col)):
                            if it2 == len(col)-1:
                                if(col[it2] == 0):
                                    col[it2] = col[it2-1]
                                    all_frames_right_hand[it2][it] = col[it2-1]
                            else:
                                if(col[it2] == 0):
                                    if(col[it2+1] == 0):
                                        col[it2] = col[it2-1]
                                        all_frames_right_hand[it2][it] = col[it2-1]
                                    else:
                                        col[it2] = (col[it2-1] + col[it2+1])/2
                                        all_frames_right_hand[it2][it] = (col[it2-1] + col[it2+1])/2
                        
                        # wyliczamy wariancje   
                        srednio =  sum(col)/sequence_length
                        varations.append(np.sum( (col - srednio)**2 ) / len(col))
                        col = []
                    # decycja na podstawie wariancji    
                    if(np.sum(varations) >= 0.8):
                        print("suma: ", np.sum(varations), "prawa dlon sie porusza")
                        s2 = True
                    else:
                        print("suma: ", np.sum(varations), "prawa dlon sie nie poruszala")
                        
                    # # eliminacja zer w wektorze pose
                    col = []
                    for it in range(8):
                        for frame in all_frames_pose_right:
                            col.append(frame[it])   # 1 dobrze wyznacza

                        for it2 in range(1, len(col)):
                            if it2 == len(col)-1:
                                if(col[it2] == 0):
                                    col[it2] = col[it2-1]
                                    all_frames_pose_right[it2][it] = col[it2-1]
                            else:
                                if(col[it2] == 0):
                                    if(col[it2+1] == 0):
                                        col[it2] = col[it2-1]
                                        all_frames_pose_right[it2][it] = col[it2-1]
                                    else:
                                        col[it2] = (col[it2-1] + col[it2+1])/2
                                        all_frames_pose_right[it2][it] = (col[it2-1] + col[it2+1])/2
                        col = []
                else:
                    print("prawa reka jest podziurawiona albo nie ma jej w kadrze")
                
                
                # Decyzja czy to jedno czy 2 reczny znak
                if (s1 and s2):
                    print("do sieci na 2 rece")   
                    for i in range(sequence_length):
                        keypoints = np.concatenate([all_frames_pose_left[i], all_frames_left_hand[i], all_frames_pose_right[i], all_frames_right_hand[i]])                                
                        # Export keypoints
                        npy_path = os.path.join(DATA_PATH, action, str(sequence), str(i+1))
                        np.save(npy_path, keypoints) 
                    
                    print()
                    fixing = False
                    
                elif(s1 or s2):
                    print("do sieci dla 1 reki")
                    if s1:
                        for i in range(sequence_length):                  
                            keypoints = np.concatenate([all_frames_pose_left[i], all_frames_left_hand[i]])
                            # Export keypoints
                            npy_path = os.path.join(DATA_PATH, action, str(sequence), str(i+1))
                            np.save(npy_path, keypoints)
                    else:
                        for i in range(sequence_length):
                            keypoints = np.concatenate([all_frames_pose_right[i], all_frames_right_hand[i]])
                            # Export keypoints
                            npy_path = os.path.join(DATA_PATH, action, str(sequence), str(i+1))
                            np.save(npy_path, keypoints)
                    print()
                    fixing = False
                    
                else: #ten else (patrz opis ponizej w princie) - w przypadku real life nic nie wypisywac az data nie bedzie sie zgadzac
                    print("rece byly ale sie nie ruszaly albo jedna reka stala a z drugiej nie zebrano dobrze landmakow")
                    print("naprawiamy")
                    print()
                    shutil.rmtree(os.path.join(DATA_PATH, action, str(sequence)))

                    tmp = image.copy()
                    cv2.putText(tmp, "rece sa ale nic nie pokazuja :<", (100,200), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0 , 0, 255), 4, cv2.LINE_AA)
                    cv2.putText(tmp, "trzeba powtorzyc nagranie {}".format(sequence), (100,250), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0 , 0, 255), 4, cv2.LINE_AA)
                    cv2.putText(tmp, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, cv2.LINE_AA)
                    cv2.putText(tmp, "skip 'z'", (15,30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV date_set', tmp)
                    cv2.waitKey(2000)
                    sequence -= 1
                    fixing = True              
            
    cap.release()
    cv2.destroyAllWindows()  

hello 1
CONTINUE
hello 6
DELETE
hello 1
blank_pose_left: 0  blank_left_hand: 0   blank_pose_right: 0  blank_right_hand: 0
suma:  7.259031253021367 lewa dlon sie porusza
suma:  5.596358204266075 prawa dlon sie porusza
do sieci na 2 rece

blank_pose_left: 0  blank_left_hand: 0   blank_pose_right: 0  blank_right_hand: 0
suma:  0.040057700937051174 lewa dlon sie nie poruszala
suma:  0.02692631477819328 prawa dlon sie nie poruszala
rece byly ale sie nie ruszaly albo jedna reka stala a druga poruszala sie tak szybko ze nie zebrano dobrze landmakow
naprawiamy

blank_pose_left: 0  blank_left_hand: 0   blank_pose_right: 0  blank_right_hand: 0
suma:  0.01626071295508992 lewa dlon sie nie poruszala
suma:  0.02880873858164092 prawa dlon sie nie poruszala
rece byly ale sie nie ruszaly albo jedna reka stala a druga poruszala sie tak szybko ze nie zebrano dobrze landmakow
naprawiamy

za malo landmarkow
naprawiamy

Nie zlapano pierwszej klatki
naprawiamy

Nie zlapano pierwszej klatki
naprawiamy

Nie z